In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune
import scipy
from scipy.stats import norm

from fractions import Fraction

import models
import data_generation
import evaluation

import os

import ray
ray.init(num_cpus=25)


2023-10-11 10:46:04,473	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.9.13
Ray version:,2.0.0


## nonlinear-heterogeneity

In [3]:
beta_1=np.append(np.array([-4,2,6,-8,8,-6,6,3,-4,6,4,-3,0,0,0]),np.zeros(485))

beta_2=np.append(np.array([4,-3,2,6,-4.5,-6,4.5,6,-3,4.5,0,0,-3,4.5,0]),np.zeros(485))

beta_3=np.append(np.array([-2,2,4,-4,3,1,-3,3,-2.5,2,0,0,0,0,2]),np.zeros(485))


beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)


n1=150
n2=150
n3=150

p=beta_1.shape[0]
corval=0.5


cos_index=[10,11,12,13,14]
sin_index=[1,2,3,4,5]
square_index=[7,8]

## M1-s1

In [ ]:
method="M1"
corval=0.5
prior=[0,1,2,3,4]
vs = np.zeros((1,beta_1.shape[0]))
p = beta_1.shape[0]

In [ ]:
for i in range(1):
 
    data_generation.setup_seed(i)
    x1,y1=data_generation.generate_data_nolinear(corval,beta_1,n1,p,sin_index,cos_index,square_index)
    x2,y2=data_generation.generate_data_nolinear(corval,beta_2,n2,p,sin_index,cos_index,square_index)
    x3,y3=data_generation.generate_data_nolinear(corval,beta_3,n3,p,sin_index,cos_index,square_index)
    
    
    train_x1,train_y1 =  x1,y1
    train_x2,train_y2 =  x2,y2
    train_x3,train_y3 =  x3,y3
    
    test_x1,test_y1 = data_generation.generate_data_nolinear(corval,beta_1,10000,p,sin_index,cos_index,square_index)
    test_x2,test_y2 = data_generation.generate_data_nolinear(corval,beta_2,10000,p,sin_index,cos_index,square_index)
    test_x3,test_y3 = data_generation.generate_data_nolinear(corval,beta_3,10000,p,sin_index,cos_index,square_index)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
    
    
    ds_1_train_idx_list,ds_1_test_idx_list,ds_2_train_idx_list,ds_2_test_idx_list,ds_3_train_idx_list,ds_3_test_idx_list = data_generation.Kfold(inputs_1,inputs_2,inputs_3,5)
################## first step######################################  
    #set prior information here.
    prior=[0,1,2,3,4]
# choose lambda1,lambda2 by ray tune

    config={
            "lambda1": tune.choice([0.055,0.05,0.06]),
            "lambda2": tune.choice([0.4,0.5,0.3]),
            "alpha": tune.choice([0.003]),
            "lr":tune.choice([0.6,0.7,0.8]),
            "ga":tune.choice([0.5]),
            "inputs_1":train_x1,
            "inputs_2":train_x2,
            "inputs_3":train_x3,
           "targets_1":train_y1,
           "targets_2":train_y2,
           "targets_3":train_y3,
            "ds_1_train_idx_list":ds_1_train_idx_list,
            "ds_1_test_idx_list":ds_1_test_idx_list,
            "ds_2_train_idx_list":ds_2_train_idx_list,
            "ds_2_test_idx_list":ds_2_test_idx_list,
            "ds_3_train_idx_list":ds_3_train_idx_list,
            "ds_3_test_idx_list":ds_3_test_idx_list,
            "prior":prior
        }

    result = tune.run(
            models.train_model_main_prior,
            config=config,
            num_samples=40)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    
################## record the parameter ########################################    
    
    y_prior_1,y_prior_2,y_prior_3, prior_model_1, prior_model_2,prior_model_3=models.main_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["alpha"],final_lambda["lr"],final_lambda["ga"],
                           inputs_1,inputs_2,inputs_3,targets_1,targets_2,targets_3,prior) 
    
################## third step##################
    no_prior=[]

    config={
                "lambda1": tune.choice([0.06,0.07,0.08]),
                "lambda2": tune.choice([0.3,0.4,0.5]),
                "alpha": tune.choice([0.003]),
                "lr":tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                "ga":tune.choice([1]),
                "inputs_1":train_x1,
                "inputs_2":train_x2,
                "inputs_3":train_x3,
                "train_y1":train_y1,
                "train_y2":train_y2,
                "train_y3":train_y3,
                "y_prior_1":y_prior_1,
                "y_prior_2":y_prior_2,
                "y_prior_3":y_prior_3,
                "prior":no_prior,
                "ds_1_train_idx_list":ds_1_train_idx_list,
                "ds_1_test_idx_list":ds_1_test_idx_list,
                "ds_2_train_idx_list":ds_2_train_idx_list,
                "ds_2_test_idx_list":ds_2_test_idx_list,
                "ds_3_train_idx_list":ds_3_train_idx_list,
                "ds_3_test_idx_list":ds_3_test_idx_list,
            }

    result = tune.run(
            models.train_model_main_final,  
            config=config,
            num_samples=300)

################## fourth step##################

    #no prior
    no_prior=[]

    # set hyperparamters
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
           
    prediction,target,model_1_weight,model_2_weight,model_3_weight=models.main_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["alpha"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       inputs_1,inputs_2,inputs_3,train_y1,train_y2,train_y3,y_prior_1,y_prior_2,y_prior_3,
                       test_inputs_1,test_inputs_2,test_inputs_3,test_targets_1,test_targets_2,test_targets_3,no_prior)
    
####################################   prediction ####################################      

    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM = math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
####################################   variable selection ####################################     
    TP,FP,FN,TN=evaluation.variable_selection_hetero_inte(0.1,model_1_weight,model_2_weight,model_3_weight,beta_1,beta_2,beta_3,p)

    vsSEN=TP/(TP+FN)
    vsSPE=(TN/(TN+FP))
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=(FP+FN)/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    print(final_matrix,TPR,TNR,GM)
    print(vsSEN,vsSPE,vsGM)
    
    log_file_name = "{}.csv".format("S1_M1_corr")
    # your log path
    log_dir =
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)
        
